In [1]:
import os
import pickle
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

In [2]:
# Name and unit catalog for each variable
var_info = {
    "wind_speed": ("Wind speed", "m/s", (0,14)),
    "current_speed": ("Current speed", "m/s", (0,0.4)),
    "total_precipitation": ("Total precipitation", "m", (0,0.12)),
    "wave_height": ("Wave height", "m", (0,3)),
    "wave_period": ("Wave period", "s", (0,14.5)),

}
# select the stations you want to plot. /!\ You can only choose stations for one hydrophone
hydrophones = ["h1", "h2", "h3", "h4"]
variables = ["wind_speed", "current_speed", "total_precipitation", "wave_height", "wave_period"]

In [14]:
# Spectrogram plot
for h in hydrophones:
    fig, axs = plt.subplots(2, 3, figsize=(24, 10), sharex=False, sharey=True)
    axs = axs.flatten()

    for i, var in enumerate(variables):
        with open(f"../../../../data/test_correlation/data_soundscape/spectro_and_corr/{var}-{h}.pkl", "rb") as f:
            data = pickle.load(f)
        psd_reindexed = data["psd"]
        variable_reindexed = data["variable"]
        time= data["time"]
        title_modifier = data["title"]

        var_name, unit, scale = var_info.get(var, ("Unknown variable", "", (0,1)))

        ax1 = axs[i]
        ax2 = ax1.twinx()

        # Spectrogram
        sp = ax1.imshow(
            10*np.log10(psd_reindexed.T + 1e-12),
            aspect="auto",
            origin="lower",
            cmap="viridis",
            extent=(time[0], time[-1], psd_reindexed.columns[0], psd_reindexed.columns[-1]),
            vmin=10, vmax=95
        )
        # Overlay variable
        ax2.plot(
            time,
            variable_reindexed.values,
            color="red",
            alpha=0.6,
            linewidth=0.5
        )
        ax1.grid(False)
        ax2.grid(False)
        ax1.xaxis.grid(color="gray", linestyle="--", alpha=0.5, zorder=1)

        # Ticks and labels
        if i% 3 == 0: # left column
            ax1.set_ylabel("Frequency (Hz)")
        ax2.set_ylabel(f"{var_name} ({unit})", labelpad=10)
        ax2.set_ylim(scale)

        #Formatting time labels (only on bottom row)
        ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))
        ax1.xaxis.set_minor_locator(mdates.MonthLocator(bymonth=range(1, 13)))
        ax1.tick_params(axis='x', which='major', direction='out', bottom=True, top=False, length=10, width=1)
        ax1.tick_params(axis='x', which='minor', direction='out', bottom=True, top=False, length=5, width=0.5)
        ax1.set_title(f"{var_name} {title_modifier}")

        if i >= 2:
            ax1.set_xlabel("Date")
        else:
            ax1.xaxis.set_ticklabels([])

    fig.delaxes(axs[5])
    fig.suptitle(f"Spectrogram with overlaid variable - {h}", fontsize=16)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95]) # leave space for subtitle
    fig.subplots_adjust(wspace=0.2) # horizontal space between figures

    # Save figure
    output_path = f"/home/imonge/Documents/Abiotic/{h}/spectro_all"
    os.makedirs(output_path, exist_ok=True)
    fig.savefig(os.path.join(output_path, f"spectrogram_{h}.png"))
    plt.close(fig)

In [3]:
# Correlations plot
for h in hydrophones:
    fig, axs = plt.subplots(2, 3, figsize=(20, 10), sharex=True, sharey=True)
    axs = axs.flatten()

    for i, var in enumerate(variables):
        with open(f"../../../../data/test_correlation/data_soundscape/spectro_and_corr/{var}-{h}.pkl", "rb") as f:
            data = pickle.load(f)
        frequencies = data["frequencies"]
        correlations = data["correlations"]

        var_name, unit, scale = var_info.get(var, ("Unknown variable", "", (0,1)))

        ax = axs[i]
        ax.plot(frequencies, correlations, color="steelblue", linewidth=0.5)
        ax.axhline(0, color='gray', linestyle="--", linewidth=1, alpha=0.5)

        ax.set_yticks([round(t,1) for t in np.arange(-1, 1.01, 0.1)])
        ax.tick_params(axis='y', labelleft=True)
        ax.set_ylim(-0.4, 0.75)

        if i >= 2: # bottom row
            ax.set_xlabel("Frequency (Hz)")
            ax.tick_params(axis='x', labelbottom=True)
        else:
            ax.tick_params(axis='x', labelbottom=False)

        ax.set_title(f"{var_name}", fontsize=12)

    fig.delaxes(axs[5])
    fig.suptitle(f"Spearman's correlations between PSD median amplitude and variable - {h}", fontsize=16)
    fig.text(0.06, 0.5, "Spearman's correlation coefficient", va='center', rotation='vertical', fontsize=12)

    fig.tight_layout(rect=[0.08, 0.03, 1, 0.95]) # leave space for subtitle
    fig.subplots_adjust(wspace=0.2)

    # Save figure
    output_path = f"/home/imonge/Documents/Abiotic/{h}/corr_all"
    os.makedirs(output_path, exist_ok=True)
    fig.savefig(os.path.join(output_path, f"spearman_corr_{h}.png"))
    plt.close(fig)

/tmp/ipykernel_11790/3326934637.py:38: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
